
# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

国勢調査の小地域集計データ（人口構成、労働力構成、住宅形態など）から、その地域の収入を推定することが考えられます。推定のための機械学習・深層学習モデルを構築しなさい。


- データを観察・理解する上で、データの構造を説明しながら、適切なデータ整形を行いなさい
- データ構造や分析結果に対して、少なくとも二つの図で可視化を行いなさい
- モデルの精度を評価し、できるだけ精度が高いモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめなさい


In [10]:
import pandas as pd
import numpy as np

# 国勢調査データのファイル名をリストにまとめる
census_files = [
    'h27_age_df.csv',
    'h27_family_df.csv',
    'h27_work_status_df.csv',
    'h27_marriage_df.csv',
    'h27_labor_df.csv',
    'h27_job_df.csv',
    'h27_indusry_df.csv',
    'h27_house_info_df.csv',
    'h27_house_df.csv',
    'h27_gender_df2.csv'
]

# 共通して含まれる列名を結合キーとする
merge_keys = ['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name']

# ベースとなる最初データフレームを読み込み、不要な列を削除する
df_merged = pd.read_csv(census_files[0])
df_merged.drop(columns=['state_name', 'city_name', 'district_name', 'district2_name'], errors='ignore', inplace=True)

# 順次、他のデータフレームを結合する
for file in census_files[1:]:
    df_to_merge = pd.read_csv(file)
    
    # 重複する列を削除する
    df_to_merge.drop(columns=['state_name', 'city_name', 'district_name', 'district2_name'], errors='ignore', inplace=True)

    # 結合キーを特定（district_id, district2_id, level_identifierを共通の列とする）
    common_cols = list(set(df_merged.columns) & set(df_to_merge.columns))
    
    # 結合の実行
    df_merged = pd.merge(df_merged, df_to_merge,
                         on=common_cols,
                         how='outer',
                         suffixes=('', f'_{file.replace(".csv", "")}'))
    
    print(f"'{file}' を結合しました。結合後のデータ形状: {df_merged.shape}")

# データクレンジング
def clean_and_convert(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            # '-'や'X'を'0'に変換
            df[col] = df[col].astype(str).str.replace('-', '0').str.replace('X', '0')
            try:
                # object型の列を数値型に変換
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                continue
    return df

# データクレンジングを実行する
df_merged = clean_and_convert(df_merged)

# 求めたい、income_meanを読み込む
df_income = pd.read_csv('income_district.csv')

# income_district.csv との結合
# district_idとarea_codeで結合
final_df = pd.merge(df_income, df_merged,
                    left_on='area_code', right_on='district_id',
                    how='inner')

# 最終的なデータフレームの形状と情報を表示
print("\n最終的なデータフレームの結合と整形が完了しました。")
print(f"最終的なデータフレームの形状: {final_df.shape}")
print("最終的なデータフレームの最初の5行:")
print(final_df.head())
print("\n最終的なデータフレームの情報:")
print(final_df.info())

# 最終データをCSVファイルに出力
final_df.to_csv('merged_census_income_data.csv', index=False)
print("\n'merged_census_income_data.csv'としてファイルを保存しました。")

'h27_family_df.csv' を結合しました。結合後のデータ形状: (251258, 74)
'h27_work_status_df.csv' を結合しました。結合後のデータ形状: (251258, 82)
'h27_marriage_df.csv' を結合しました。結合後のデータ形状: (251258, 91)
'h27_labor_df.csv' を結合しました。結合後のデータ形状: (251258, 95)
'h27_job_df.csv' を結合しました。結合後のデータ形状: (251258, 119)
'h27_indusry_df.csv' を結合しました。結合後のデータ形状: (251258, 161)
'h27_house_info_df.csv' を結合しました。結合後のデータ形状: (251258, 175)
'h27_house_df.csv' を結合しました。結合後のデータ形状: (251258, 188)
'h27_gender_df2.csv' を結合しました。結合後のデータ形状: (251258, 191)

最終的なデータフレームの結合と整形が完了しました。
最終的なデータフレームの形状: (228262, 193)
最終的なデータフレームの最初の5行:
   area_code  income_mean  district_id  district2_id  level_identifier  10-14  \
0      13228   480.984419        13228          10.0                 2    238   
1      13228   480.984419        13228          20.0                 2    336   
2      13228   480.984419        13228          30.0                 2    109   
3      13228   480.984419        13228          40.0                 2     88   
4      13228   480.984419        13228

In [11]:
#データの構造
 #income_district.csv
  #area_code：地域の識別コード、income_mean：地域の平均収入
 #h27_から始まるデータ
  #年齢構成や家族構成や就業状況などの地域の様々な特徴を表す
  #district_id, district2_id, level_identifier, state_name, city_name, district_name, district2_nameを共通の識別子とする
 #データ型
   #人口数や世帯数などの数値であるはずの列がobject型になっている
 #データの大きさ
   #国勢調査は収入データより小さい地域単位が集計されている

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Windows環境で日本語フォントを設定する
plt.rcParams['font.family'] = 'Yu Gothic'
plt.rcParams['font.size'] = 12

# 結合済みのデータセットを読み込む
df = pd.read_csv('merged_census_income_data.csv')

# 1.平均収入のヒストグラムを作成 ---
plt.figure(figsize=(10, 6))
plt.hist(df['income_mean'], bins=30, edgecolor='black', alpha=0.7)
plt.title('平均収入の分布')
plt.xlabel('平均収入')
plt.ylabel('度数')
plt.grid(axis='y', alpha=0.5)
plt.tight_layout()
plt.savefig('income_mean_histogram.png')
print("ヒストグラム 'income_mean_histogram.png' を作成しました。")
plt.close()

# 2.平均収入と世帯人数の散布図を作成 ---
# 'family_member_count' 列が存在するか確認
if 'family_member_count' in df.columns:
    plt.figure(figsize=(10, 6))
    plt.scatter(df['family_member_count'], df['income_mean'], alpha=0.5)
    plt.title('平均収入 vs. 世帯人数')
    plt.xlabel('世帯人数 (family_member_count)')
    plt.ylabel('平均収入 (income_mean)')
    plt.grid(True, alpha=0.5)
    plt.tight_layout()
    plt.savefig('income_vs_familymembercount_scatterplot.png')
    print("散布図 'income_vs_familymembercount_scatterplot.png' を作成しました。")
    plt.close()
else:
    print("エラー: 'family_member_count' 列が見つかりませんでした。")


ヒストグラム 'income_mean_histogram.png' を作成しました。
散布図 'income_vs_familymembercount_scatterplot.png' を作成しました。


In [20]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\sarry\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#まずは最も簡単な単回帰モデルで考える。
#すべての国勢調査データを整形・結合したものを説明変数、その地域の平均収入を目的変数とする
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# データを読み込む
df = pd.read_csv('merged_census_income_data.csv')

# 説明変数(X) と目的変数 (y) を設定する
# income_mean、area_code、district_id を除いたすべての数値を説明変数とする
X = df.drop(columns=['income_mean', 'area_code', 'district_id', 'district2_id', 'level_identifier'])
y = df['income_mean']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 線形回帰モデルのインスタンスを作成し、訓練
model = LinearRegression()
model.fit(X_train, y_train)

# テストデータで予測
y_pred = model.predict(X_test)

# モデルの評価
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("--- モデルの評価結果 ---")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R^2 (決定係数): {r2:.2f}")

# 予測結果の一部を表示
print("\n--- 予測結果の比較（一部） ---")
comparison_df = pd.DataFrame({'実際の値': y_test, '予測値': y_pred})
print(comparison_df.head())

--- モデルの評価結果 ---
RMSE (Root Mean Squared Error): 56.38
R^2 (決定係数): 0.01

--- 予測結果の比較（一部） ---
              実際の値         予測値
151555  494.283808  453.149058
126512  433.162791  451.140980
208993  491.389238  449.705568
82447   502.368141  439.246309
143255  461.107011  456.183339


In [ ]:
#決定係数は、0.01と非常に低く、このモデルでは平均収入をほとんど説明できていない。
#RSMEは56.23と非常に大きく、誤差が大きいことがわかる。予測が大きく外れていると考えられる。